# 数据预处理

将数据放在 `data/raw` 目录下：

In [1]:
from pathlib import Path
import zipfile
import pandas as pd
import numpy as np

In [2]:
root = Path('data')
[x for x in (root/'raw').iterdir()]

[WindowsPath('data/raw/aichallenge2019_移动广告反欺诈算法挑战赛-0621.zip'),
 WindowsPath('data/raw/aichallenge2019_移动广告反欺诈算法挑战赛提交样例.xls')]

In [3]:
submit = pd.read_csv(list((root/'raw').glob('*.xls'))[0])
submit

,sid,label
0,4601f528-865c-4238-a43f-cf4a4e11027d-156017418...,1
1,c727e4bf-2ae1-4829-9aa1-730277ee2042-156013863...,0
2,fe23d40a-0db8-403f-8df0-6e8b9ea461c3-156009638...,1
3,308b053c-5d7c-43b8-99dc-8aad647f953f-156017652...,0
4,94477644-63cd-4f3c-9b2a-f3d25f1ec7e2-156014459...,1


In [4]:
[x for x in root.iterdir()]

[WindowsPath('data/raw')]

In [5]:
from pathlib import Path
import zipfile
import pandas as pd


def unzip(root):
    root = Path(root)
    Z = zipfile.ZipFile(list((root/'raw').glob('*.zip'))[0])
    for fn in Z.namelist():
        extract_path = Path(Z.extract(fn, root))
        dirt = root/extract_path.parts[1]  # 重命名之后的空目录
        # 防止解压后的数据中文名称乱码
        extract_path.rename(root/fn.encode('cp437').decode('gbk'))
    dirt.rmdir()  # 删除空目录
    return [x for x in root.iterdir() if x != root/'raw'][0]


class toCSV:
    def __init__(self, root):
        self.root = root

    def get_df(self, dataType, sep='\t'):
        path = list(self.root.glob(f'*{dataType}*'))[0]
        df = pd.read_csv(path, sep=sep, index_col='sid')
        return df

In [6]:
root = unzip('data')
tp = toCSV(root)

train_df = tp.get_df('train')
test_df = tp.get_df('train')
sample_df = tp.get_df('sample', ',')

In [7]:
sample_df

,label
sid,
4601f528-865c-4238-a43f-cf4a4e11027d-1560174183640,1
c727e4bf-2ae1-4829-9aa1-730277ee2042-1560138630353,0
fe23d40a-0db8-403f-8df0-6e8b9ea461c3-1560096380670,1
308b053c-5d7c-43b8-99dc-8aad647f953f-1560176523438,0
94477644-63cd-4f3c-9b2a-f3d25f1ec7e2-1560144590202,1


In [8]:
train_df.columns

Index(['label', 'pkgname', 'ver', 'adunitshowid', 'mediashowid', 'apptype',
       'nginxtime', 'ip', 'city', 'province', 'reqrealip', 'adidmd5',
       'imeimd5', 'idfamd5', 'openudidmd5', 'macmd5', 'dvctype', 'model',
       'make', 'ntt', 'carrier', 'os', 'osv', 'orientation', 'lan', 'h', 'w',
       'ppi'],
      dtype='object')

In [9]:
int_cols = ['label', 'apptype', 'dvctype', 'ntt', 'orientation', 'h', 'w', 'ppi']
time_cols = ['nginxtime']

In [10]:
train_df.loc[:, int_cols] = train_df.loc[:, int_cols].astype('int').astype('category')

In [11]:
train_df.dtypes

label           category
pkgname           object
ver               object
adunitshowid      object
mediashowid       object
apptype         category
nginxtime        float64
ip                object
city              object
province         float64
reqrealip         object
adidmd5           object
imeimd5           object
idfamd5           object
openudidmd5       object
macmd5            object
dvctype         category
model             object
make              object
ntt             category
carrier          float64
os                object
osv               object
orientation     category
lan               object
h               category
w               category
ppi             category
dtype: object